# Chat Completions

Exploring chat completions using llama-jax.

# Setup

In [1]:
from time import time_ns as seed

from jax import random

import llama_jax as ll

Metal device set to: Apple M1 Max

systemMemory: 64.00 GB
maxCacheSize: 24.00 GB



W0000 00:00:1741024535.903599 11875102 mps_client.cc:510] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!
I0000 00:00:1741024535.914191 11875102 service.cc:145] XLA service 0x1187adf90 initialized for platform METAL (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741024535.914201 11875102 service.cc:153]   StreamExecutor device (0): Metal, <undefined>
I0000 00:00:1741024535.915357 11875102 mps_client.cc:406] Using Simple allocator.
I0000 00:00:1741024535.915364 11875102 mps_client.cc:384] XLA backend will use up to 51539214336 bytes on device 0 for SimpleAllocator.


In [2]:
# Parameters
max_tokens = 30

In [3]:
key = random.key(seed())

# Configure chat session
session = ll.chat.session(warmup_tokens=20)

Warmup (1/10):   0%|          | 0/20 [00:00<?, ?it/s]

Warmup (2/10):   0%|          | 0/20 [00:00<?, ?it/s]

Warmup (3/10):   0%|          | 0/20 [00:00<?, ?it/s]

Warmup (4/10):   0%|          | 0/20 [00:00<?, ?it/s]

Warmup (5/10):   0%|          | 0/20 [00:00<?, ?it/s]

Warmup (6/10):   0%|          | 0/20 [00:00<?, ?it/s]

Warmup (7/10):   0%|          | 0/20 [00:00<?, ?it/s]

Warmup (8/10):   0%|          | 0/20 [00:00<?, ?it/s]

Warmup (9/10):   0%|          | 0/20 [00:00<?, ?it/s]

Warmup (10/10):   0%|          | 0/20 [00:00<?, ?it/s]

# Prompt 0: Trains

In [4]:
prompt = "Why do you like traveling by train?"

In [5]:
key, subkey = random.split(key)

with ll.render.token_view(session.config, prompt=prompt) as tv:
    for token in ll.chat.complete(session, content=prompt, key=subkey, max_tokens=max_tokens):
        tv.add_token(token)

Output()

# Prompt 1: Paris

In [6]:
prompt = "Where do you go when you visit Paris?"

In [7]:
key, subkey = random.split(key)

with ll.render.token_view(session.config, prompt=prompt) as tv:
    for token in ll.chat.complete(session, content=prompt, key=subkey, max_tokens=max_tokens):
        tv.add_token(token)

Output()

# Prompt 2: Once Upon a Time

In [8]:
prompt = "Tell me a story."

In [9]:
key, subkey = random.split(key)

with ll.render.token_view(session.config, prompt=prompt) as tv:
    for token in ll.chat.complete(session, content=prompt, key=subkey, max_tokens=max_tokens):
        tv.add_token(token)

Output()